In [1]:
import argparse
import os
import time

import PIL
from PIL import Image

import numpy as np
import torchvision
import pickle

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:

vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16.eval()

class VGG16relu7(nn.Module):
    def __init__(self):
        super(VGG16relu7, self).__init__()
        self.features = nn.Sequential( *list(vgg16.features.children()))
    # garder une partie du classifieur, -2 pour s'arrêter à relu7
        self.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-2])
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def preprocess(x):
    mean = [0.485,0.456,0.406]
    std = [0.229,0.224,0.225]
    return transforms.functional.normalize(x, mean, std, False)

def predict(img): 
    img = img.resize((224, 224), Image.BILINEAR)
    img = np.array(img, dtype=np.float32) / 255
    img = img.transpose((2, 0, 1))
    img = np.expand_dims(img, 0)
    x = torch.Tensor(img)
    x = preprocess(x)

    y = vgg16(x)
    y = y.detach().cpu().numpy() # transformation en array numpy
    yargmax = np.argmax(y)
    return imagenet_classes[yargmax]


def predict_and_show(img_path):
    img = Image.open(img_path)
    plt.figure()
    plt.imshow(img)
    classe = predict(img)
    print("Classe prédite : "+str(classe))


def extract_features(data, model):
    #####################
    ## Votre code ici  ##
    #####################
    # init features matrices
    #X = np.zeros((len(data), 4096)) 
    #X = np.zeros((len(data), 512)) # Resnet18
    X = np.zeros((len(data), 1024)) # Googlenet
    y = np.zeros((len(data)))
    ####################
    ##      FIN        #
    ####################

    for i, (input, target) in enumerate(data):
        if i % PRINT_INTERVAL == 0:
            print('Batch {0:03d}/{1:03d}'.format(i, len(data)))
        if CUDA:
            input = input.cuda()
        #####################
        ## Votre code ici  ##
        #####################
        # Feature extraction à faire
        out = model(input)
        x_feat = out.detach().cpu().numpy()
        x_feat = x_feat / np.linalg.norm(x_feat, 2, 1, True)
        X = np.append(X, x_feat, axis = 0)
        y = np.append(y, target.detach().numpy())
        ####################
        ##      FIN        #
        ####################    

    return X, y

def main(path="15SceneData", batch_size=8):
    print('Instanciation de VGG16')
    vgg16 = models.vgg16(pretrained=True)

    print('Instanciation de VGG16relu7')
    #####################
    ## Votre code ici  ##
    #####################
    # Remplacer par le modèle par un réseau tronqué pour faire de la feature extraction
    # On créera une nouvelle classe VGG16relu7 ici
    #model =  VGG16relu7()
    #model = resnet18cut()
    model = VGG16relu7()
    ####################
    ##      FIN        #
    ####################      
    
    model.eval()
    if CUDA: # si on fait du GPU, passage en CUDA
        cudnn.benchmark = True
        model = model.cuda()

    # On récupère les données
    print('Récupération des données')
    train, test = get_dataset(batch_size, path)

    # Extraction des features
    print('Feature extraction')
    
    X_train, y_train = extract_features(train, model)
    print("X shape : "+str(X_train.shape))
    X_test, y_test = extract_features(test, model)

    #####################
    ## Votre code ici  ##
    #####################
    # Apprentissage et évaluation des SVM à faire
    print('Apprentissage des SVM')
    from sklearn.svm import LinearSVC
    from sklearn.metrics import accuracy_score
    svm = LinearSVC(C=1.0)
    svm.fit(X_train,y_train)
    y_predict = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict)
    ####################
    ##      FIN        #
    ####################    
    print('Accuracy = %f' % accuracy)
